In [0]:
PROJECT_PATH = '~/project/ga_isochrones'
ISOCHONES_PATH = '$ISOCHRONES_DATA/dartmouth/import/afep0_cfht_sdss_hsc'

MAG1, MAG2, MAG3 = 'hsc_g', 'hsc_i', 'hsc_g'

In [0]:
%matplotlib inline

In [0]:
import os, sys
import numpy as np
import h5py as h5
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

In [0]:
# Disable GPU access for TensorFlow, use CPU instead.
# Use this option if using shared GPUs on big servers to prevent holding the GPU memory
# when notebooks are left running for a longer time. Do not use this option when
# high performance execution is required.
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [0]:
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()

In [0]:
tf.config.list_physical_devices('CPU')

In [0]:
tf.config.list_physical_devices('GPU') 

In [0]:
os.getcwd(), os.environ['PYTHONPATH']

# Evaluate isochrone grid

## Load grid from HDF5

In [0]:
from pfs.ga.isochrones import IsoGrid

In [0]:
iso = IsoGrid()
iso.load(os.path.join(os.path.expandvars(ISOCHONES_PATH), 'isochrones.h5'))

In [0]:
iso.axes.keys()

In [0]:
for k in iso.values.keys():
    print(k, 
          tf.math.count_nonzero(tf.math.is_inf(iso.values[k])).numpy(),
          tf.math.count_nonzero(tf.math.is_nan(iso.values[k])).numpy())

## Atmospheric parameter ranges

In [0]:
def get_range(k):
    mask = ~tf.math.is_inf(iso.values[k])
    mn = tf.math.reduce_min(tf.where(mask, iso.values[k], 1e10))
    mx = tf.math.reduce_max(tf.where(mask, iso.values[k], 0))
    return mn.numpy(), mx.numpy()
    
log_t_eff_min, log_t_eff_max = get_range('Log_T_eff')
print('T_eff', 10 ** log_t_eff_min, 10 ** log_t_eff_max)
print('log_g', get_range('log_g'))
print('[Fe/H]', iso.Fe_H[0].numpy(), iso.Fe_H[-1].numpy())

## Plot the isochrones

In [0]:
iso.Fe_H

In [0]:
iso.log_t

In [0]:
iso.EEP

In [0]:
f, ax = plt.subplots(1, 1, figsize=(6, 6), dpi=120)

X = (iso.values[MAG1] - iso.values[MAG2]).numpy().flatten()
Y = iso.values[MAG3].numpy().flatten()

ax.plot(X, Y, 'sk', ms=0.1, alpha=0.1, rasterized=True)
ax.invert_yaxis()

## Plot a few isochrones

In [0]:
iso.values[MAG1].shape

In [0]:
iso._ip._find_nearby(iso.log_t, tf.convert_to_tensor(9.1, dtype=tf.float64))

In [0]:
s = np.s_[1, 16, :]

plt.figure(figsize=(5, 5), facecolor='w', dpi=120)

m1 = iso.values[MAG1][s]
m2 = iso.values[MAG2][s]

plt.plot(m1 - m2, m1, '-')

m1 = iso.values[MAG1][s]
m2 = iso.values[MAG2][s]

plt.plot(m1 - m2, m1, '-')

plt.xlabel(f'${MAG1} - {MAG2}$')
plt.ylabel(f'${MAG3}$')
plt.ylim(14, -5)

plt.grid(True)

## Plot parameter coverage

In [0]:
value = iso.values[MAG1].numpy()
index = np.where(~np.isnan(value) & ~np.isinf(value))
index

In [0]:
iso.Fe_H

In [0]:
plt.figure(figsize=(6, 6), dpi=96, facecolor='w')
plt.plot(iso.Fe_H.numpy()[index[0]], iso.log_t.numpy()[index[1]], '.', alpha=0.01)
plt.xlabel('[Fe/H]')
plt.ylabel('log age (Gyr)')

In [0]:
for fe_h in range(iso.Fe_H.shape[0]):
    filt = index[0] == fe_h

    plt.figure(figsize=(6, 6), dpi=96, facecolor='w')
    plt.plot(iso.log_t.numpy()[index[1][filt]], iso.EEP.numpy()[index[2][filt]], '.')
    plt.xlabel('log age (Gyr)')
    plt.ylabel('EEP')
    plt.title('[Fe/H] = {}'.format(iso.Fe_H[fe_h]))